In [1]:
# Automatically update imported Modules
%load_ext autoreload
%autoreload 2
# Required Libraries
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
from model import ConvNN
import os
from torch.nn.functional import cross_entropy, softmax
from torch.optim import Adam
from torch.utils.data import Dataset, DataLoader
from tqdm.autonotebook import tqdm
from sklearn.model_selection import train_test_split

/home/akhil/anaconda3/lib/python3.7/site-packages/tqdm/autonotebook.py:17: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


In [2]:
# Version of Pytorch
torch.__version__
# Set Seed
torch.manual_seed(42)

In [3]:
device = 'cuda'
# device = 'cpu'  # uncomment this line to run the model on the GPU

In [4]:
# Load Dataset
digit_pixels = pd.read_csv("data/train.csv")
digit_pixels.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
# Convert Data Type
## To Numpy
digit_pixels_numpy = digit_pixels.to_numpy()

# Numpy Manipulation
## Splice out first Column
labels = digit_pixels_numpy[:,0]
digit_pixels_numpy = np.delete(digit_pixels_numpy,0,1)
## Split Data into Training and Validation Set
X_train, X_test, y_train, y_test = train_test_split(digit_pixels_numpy, labels, test_size=0.2)

## Resize Array to Image Form
X_train = np.resize(X_train,(X_train.shape[0],1,28,28))

## To Tensor
X_train = torch.from_numpy(X_train)
X_test = torch.from_numpy(X_test)
y_train = torch.from_numpy(y_train)
y_test = torch.from_numpy(y_test)


X_train = X_train.type(torch.cuda.FloatTensor)


print(X_train.shape)

torch.Size([33600, 1, 28, 28])


In [6]:
# Checking for Cuda 
use_cuda = torch.cuda.is_available()
print("Cuda Available:", use_cuda)

# Setting up Model
model = ConvNN()
if use_cuda:
    model = model.cuda()

Cuda Available: True


In [7]:
# Training Parameters
epochs = 1
batch_size = 64
lr = 1e-3

optimizer = Adam(model.parameters(), lr=lr)

In [22]:
# Loss Function
def loss_function(x, y, model):
    print(x[1])
    logits = model(x)
    print(logits.shape)
    out = softmax(logits, dim=1)
    out = torch.argmax(out, dim=1)
    loss = cross_entropy(logits, y)
    return loss, logits

In [23]:
# Training
losses = []
accuracies = []
num_train_batches = int(torch.ceil(torch.tensor(len(X_train) / batch_size)).item())

## Creating Dataset Object
train = torch.utils.data.TensorDataset(X_train, y_train)

for epoch in range(epochs):
        train_loader = DataLoader(train, batch_size=batch_size, shuffle=True)
        for x,y in tqdm(iter(train_loader), total=num_train_batches):
            ##########################################################
            # YOUR CODE HERE
            x = x.to(model.device())
            y = y.to(model.device())
            x
            loss,logits = loss_function(x,y,model)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            avg_loss = loss/(x.size(0))
            losses.append(avg_loss)
            
            pred = logits.max(1, keepdim=True)[1] # get the index of the max logit value
            correct = pred.eq(y.view_as(pred)).sum().item()
            avg_correct = correct/(x.size(0))
            accuracies.append(avg_correct)

tensor([[[  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
            0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
            0.,   0.,   0.,   0.,   0.,   0.],
         [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
            0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
            0.,   0.,   0.,   0.,   0.,   0.],
         [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
            0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
            0.,   0.,   0.,   0.,   0.,   0.],
         [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
            0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
            0.,   0.,   0.,   0.,   0.,   0.],
         [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
            0.,  70., 254.,  23.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
            0.,   0.,   0.,   0.,   0.,   0.],
     

AssertionError: 

In [ ]:
# Plot of Training
fig = plt.figure(figsize=(10,3))
plt.subplot(121)
plt.plot(losses)
plt.xlabel("Iteration")
plt.ylabel("Training Loss")
plt.subplot(122)
plt.plot(accuracies)
plt.xlabel("Iteration")
plt.ylabel("Training Accuracy")
plt.show()